## Strategy:
The strategy here is very simple. You pick a threshold somewhere between 0.1 and 0.9, which is determined by the parameter pp_theshold. Then you scale all predictions greater than the threshold towards 1 and all smaller than the threshold towards 0. The degree of scaling is determined by setting the parameter pp_factor.

## Results:
This postprocessing is very parameter sensitive and parameter tuning should optimally be done by averaging optimal parameters across multiple folds in order to provide a stable result, but even without doing that this provided us with a 0.0001 boost on local CV and a .00002 boost on private LB. Unfortunately, our solution was overall not good enough for this to be valuable.

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
sub = pd.read_csv('../input/moa-final-inference-gru-b4-save/submission.csv')
oof = np.load('../input/moa-final-inference-gru-b4-save/oof_blend.npy')
tar = np.load('../input/moa-final-inference-gru-b4-save/oof_targets.npy')

In [ ]:
pp_threshold = 0.2
pp_factor = 0.0019
targets = [col for col in sub.columns if col!='sig_id']

In [ ]:
def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.mean(-logloss)

In [ ]:
mean = np.ones((len(oof), 206)) * pp_threshold
mean_test = np.ones((len(sub[sub[targets[0]]!=0.0]), 206)) * pp_threshold

In [ ]:
mean

In [ ]:
mean_test

In [ ]:
np.round(mean_logloss(oof, tar), 6)

In [ ]:
diff = oof - mean
scale = diff * pp_factor
oof += scale
oof = np.clip(oof, 0.0000000000000000001, 0.999999999999999999999999999)

In [ ]:
diff_test = sub.loc[sub[targets[0]]!=0.0, targets].values - mean_test
scale_test = diff_test * pp_factor
sub.loc[sub[targets[0]]!=0.0, targets] += scale_test
sub.loc[sub[targets[0]]!=0.0, targets] = np.clip(sub.loc[sub[targets[0]]!=0.0, targets], 0.0000000000000000001, 0.999999999999999999999999999)

In [ ]:
np.round(mean_logloss(oof, tar), 6)

In [ ]:
sub